# SDAB Melting Temperature Prediction using ML

### Load Modules and Datasets

In [1]:
### Initialize seed for reproducibility
import numpy as np
np.random.seed(0)
import math

### Data Wrangling and Plots
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 600
plt.rcParams['savefig.dpi'] = 600
import seaborn as sns

### Model Diagnostics - keep it simple and use repeated K-fold cross-validation; predictors are more important.
### We would want to evaluate the model multiple times so we can be confident about using the model.
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score, train_test_split, RepeatedKFold

In [100]:
#data = pd.read_csv("./tm_predictors/tm_dataset_nsp.csv")
data = pd.read_csv("./tm_predictors/tm_dataset_af2.csv")
#data = pd.read_csv("./tm_predictors/tm_dataset_pchars_maestro.csv")
#data = pd.read_csv("./tm_predictors/tm_dataset_ESM_650M.csv", header=None)
#data = pd.read_csv("./tm_predictors/tm_dataset_ESM_3B.csv", header=None)
#data = pd.read_csv("./tm_predictors/tm_dataset_ESM_15B.csv", header=None)
data.head()

fr1.pLDDT  cdr1.pLDDT  fr2.pLDDT  cdr2.pLDDT  fr3.pLDDT  cdr3.pLDDT  \
0  97.475909   96.821538  95.884615   96.975000  98.323421   76.051429   
1  96.706364   94.692308  96.510000   94.944000  97.970526   64.882143   
2  97.194091   96.284615  97.346923   96.206000  97.610263   97.532105   
3  96.340909   95.048462  96.642308   94.969000  96.982368   96.404211   
4  95.465455   86.233636  93.625385   84.005455  95.632105   77.029524   

   fr4.pLDDT  
0  95.653636  
1  95.596364  
2  96.727273  
3  95.750000  
4  95.796364

In [101]:
y = pd.read_excel("sdab_data.xlsx")
y = y.tm
x = data

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# check assignments - y should be Tms only
print(x)
print(y)

     fr1.pLDDT  cdr1.pLDDT  fr2.pLDDT  cdr2.pLDDT  fr3.pLDDT  cdr3.pLDDT  \
0    97.475909   96.821538  95.884615   96.975000  98.323421   76.051429   
1    96.706364   94.692308  96.510000   94.944000  97.970526   64.882143   
2    97.194091   96.284615  97.346923   96.206000  97.610263   97.532105   
3    96.340909   95.048462  96.642308   94.969000  96.982368   96.404211   
4    95.465455   86.233636  93.625385   84.005455  95.632105   77.029524   
..         ...         ...        ...         ...        ...         ...   
562  94.110455   80.072308  93.802308   90.388000  95.187568   87.695882   
563  92.017273   88.635833  95.321538   85.972857  93.052105   91.192000   
564  93.287273   80.556667  95.452308   92.018571  95.777632   91.175333   
565  95.014545   86.904615  95.586154   90.269000  96.264474   83.921176   
566  93.682727   75.377500  95.038462   87.181000  95.228684   66.904000   

     fr4.pLDDT  
0    95.653636  
1    95.596364  
2    96.727273  
3    95.750000  
4 

### ML Model 1: XGBoost

In [102]:
import xgboost as xgb
# initialize base model
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

#### diagnostic: repeated K-fold cross-validation

In [103]:
# define model evaluation method
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)

# evaluate model
scores = cross_val_score(xgb_model, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores = np.absolute(scores)
print('Mean MAE: %.2f (%.2f)' % (scores.mean(), scores.std()))


Mean MAE: 6.95 (0.44)


In [104]:
# fit the model using training values
xgb_model.fit(x_train, y_train)

xgb_predictions = xgb_model.predict(x_test)

rmse=math.sqrt(mean_squared_error(y_test, xgb_predictions))
mae=mean_absolute_error(y_test, xgb_predictions)
coeff_det=r2_score(y_test, xgb_predictions)


print(mae, rmse, coeff_det)


7.160907358712823 9.016404249164083 0.27397009561889973


### ML Model 2: Random Forest

In [105]:
from sklearn.ensemble import RandomForestRegressor
# Fitting Random Forest Regression to the dataset
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

In [106]:
# define model evaluation method
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)

# evaluate model
scores = cross_val_score(rf_model, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores = np.absolute(scores)
print('Mean MAE: %.2f (%.2f)' % (scores.mean(), scores.std()))

Mean MAE: 6.46 (0.47)


In [107]:
rf_model.fit(x_train, y_train)

rf_predictions = rf_model.predict(x_test)

rmse=math.sqrt(mean_squared_error(y_test, rf_predictions))
mae=mean_absolute_error(y_test, rf_predictions)
coeff_det=r2_score(y_test, rf_predictions)


print(mae, rmse, coeff_det)

6.664690382933333 8.264057314791243 0.39007785740288403


### ML Model 3: SVM

In [108]:
from sklearn import svm
svm_model = svm.SVR(kernel="poly")

In [109]:
# define model evaluation method
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)

# evaluate model
scores = cross_val_score(svm_model, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores = np.absolute(scores)
print('Mean MAE: %.2f (%.2f)' % (scores.mean(), scores.std()))

Mean MAE: 7.67 (0.41)


In [110]:
svm_model.fit(x_train, y_train)

svm_predictions = svm_model.predict(x_test)

rmse=math.sqrt(mean_squared_error(y_test, svm_predictions))
mae=mean_absolute_error(y_test, svm_predictions)
coeff_det=r2_score(y_test, svm_predictions)


print(mae, rmse, coeff_det)

8.308342796479025 9.935734055699402 0.11836732800609884


### ML Model 4: LASSO

In [111]:
from sklearn import linear_model
lasso_model = linear_model.Lasso(random_state=42)

In [112]:
# define model evaluation method
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)

# evaluate model
scores = cross_val_score(lasso_model, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores = np.absolute(scores)
print('Mean MAE: %.2f (%.2f)' % (scores.mean(), scores.std()))

Mean MAE: 8.05 (0.46)


In [113]:
lasso_model.fit(x_train, y_train)

lasso_predictions = lasso_model.predict(x_test)

rmse=math.sqrt(mean_squared_error(y_test, lasso_predictions))
mae=mean_absolute_error(y_test, lasso_predictions)
coeff_det=r2_score(y_test, lasso_predictions)


print(mae, rmse, coeff_det)

8.812900816268618 10.307987737065421 0.05106700931764807


## Tabulate Predictions

In [114]:
results = pd.concat([y_test.reset_index(drop=True),
                     pd.DataFrame(xgb_predictions).reset_index(drop=True),
                     pd.DataFrame(rf_predictions).reset_index(drop=True),
                     pd.DataFrame(svm_predictions).reset_index(drop=True),
                     pd.DataFrame(lasso_predictions).reset_index(drop=True)],axis=1)
results.columns = ['Actual_Tm','XGBoost_Tm','RF_Tm','SVM_Tm','LASSO_Tm']
results

Actual_Tm  XGBoost_Tm      RF_Tm     SVM_Tm   LASSO_Tm
0        55.50   61.292938  64.682912  67.177393  65.292820
1        60.40   53.618809  57.037989  67.623870  68.413901
2        59.00   61.435558  63.697096  64.450382  65.552205
3        56.20   66.544594  64.455400  64.905813  66.666374
4        56.00   59.177864  62.792500  69.567474  68.316028
..         ...         ...        ...        ...        ...
109      71.05   64.381958  65.425989  65.543945  66.457665
110      66.90   66.024719  63.693300  64.598624  66.858783
111      73.00   76.294830  76.483000  75.370633  70.198344
112      49.60   60.726868  66.599133  61.039066  65.132079
113      58.00   50.610527  56.103000  65.469700  68.381949

[114 rows x 5 columns]

In [115]:
#results.to_csv("./tm_Predictions/ML_ESM15B.csv", index=False)
#results.to_csv("./tm_Predictions/ML_ESM3B.csv", index=False)
#results.to_csv("./tm_Predictions/ML_ESM650M.csv", index=False)
#results.to_csv("./tm_Predictions/ML_NSP3.csv", index=False)
#results.to_csv("./tm_Predictions/ML_AF2.csv", index=False)
#results.to_csv("./tm_Predictions/ML_PCHARS_MAESTRO.csv", index=False)